In [49]:
class Site:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def equal(self,other):
        return self.x == other.x and self.y == other.y
    
    def debug(self):
        print(f"(x,y): ({self.x},{self.y})")

In [50]:
class QuadEdge:
    def __init__(self):
        self.Org = None
        self.Dest = None
        self.Sym = None
        self.Onext = None
        self.Oprev = None


In [56]:
#Helper function
def make_edge():
    return QuadEdge()

def splice(a,b):
    return None

def prepare_data(S):
    if len(S) < 2:
        return S
    #1. Sort by x
    #2. Sort by y
    sort_x = sorted(S, key = lambda s: s.x)
    sorted_S = sorted(sort_x, key = lambda s: s.y)
    #3. Remove duplicate points
    i = 0
    while i < len(sorted_S)-1:
        if sorted_S[i].equal(sorted_S[i+1]):
            #delete i+1
            sorted_S.pop(i+1)
        else:
            i = i +1
    return sorted_S

def ccw(s1,s2,s3):
    return True

def connect(a,b):
    return None

def left_of(a,b):
    return None

def right_of(a,b):
    return None

In [62]:
# Test prepare_data
def test_prepare_data(S,expect_S,func,msg=""):
    print(f"{msg}")
    result_S = func(S)
    #print(f"result_S: {result_S}")
    assert(len(expect_S) == len(result_S))
    for i,result in enumerate(result_S):
        assert(result.equal(expect_S[i]))
#Test case
#1.
S = [Site(0,0)]
expect_S = [Site(0,0)]
test_prepare_data(S,expect_S,prepare_data,"Test with 1 site")
#2.
S = [Site(0,0),Site(0,0)]
expect_S = [Site(0,0)]
test_prepare_data(S,expect_S,prepare_data,"Test with 2 site")

#3.
S = [Site(0,0),Site(0,0),Site(0,0)]
expect_S = [Site(0,0)]
test_prepare_data(S,expect_S,prepare_data,"Test with 3 site")

#4.
S = [Site(0,0),Site(1,1),Site(2,2)]
expect_S = [Site(0,0),Site(1,1),Site(2,2)]
test_prepare_data(S,expect_S,prepare_data,"Test with 3 different sites")

#5.
S = [Site(0,0),Site(1,3),Site(1,1),Site(1,1),Site(1,2)]
expect_S = [Site(0,0),Site(1,1),Site(1,2),Site(1,3)]
test_prepare_data(S,expect_S,prepare_data,"Test with 4 different sites, 1 duplicate")

#6.
S = [Site(0,0),Site(1,3),Site(1,1),Site(1,1),Site(1,2),Site(-2,-6),Site(-2,-7)]
expect_S = [Site(-2,-7),Site(-2,-6),Site(0,0),Site(1,1),Site(1,2),Site(1,3)]
test_prepare_data(S,expect_S,prepare_data,"Test with 6 different sites, 1 duplicate")

Test with 1 site
Test with 2 site
Test with 3 site
Test with 3 different sites
Test with 4 different sites, 1 duplicate
Test with 6 different sites, 1 duplicate


In [35]:
#Divide and conquer


def delaunay_dc(S):
    length_S = len(S)
    if length_S == 2:
        s1,s2 = S[0],S[1]
        a = make_edge()
        a.Org = s1
        a.dest = s2
        return (a,a.Sym)
    elif length_S == 3:
        s1,s2,s3 = S[0],S[1],S[2]
        # Create edges a connecting s1 to s2 
        # and b connecting s2 to s3
        a = make_edge()
        b = make_edge()
        splice(a.Sym,b)
        a.Org = s1
        a.Dest = b.Org = s2
        b.Dest = s3
        #Close the triangle
        if ccw(s1,s2,s3):
            c = connect(b,a)
            return (a,b.Sym)
        elif ccw(s1,s3,s2):
            c = connect(b,a)
            return (c.Sym,c)
        else:
            return (a,b.Sym)
    elif length_S >= 4:
        L,R = S[:length_S//2],S[length_S//2:]
        ldo,ldi = delaunay_dc(L)
        rdi,rdo = delaunay_dc(R)
        #Compute the lower common tangent of L and R
        while True:
            if left_of(rdi.Org,ldi):
                ldi = ldi.Lnext
            elif right_of(ldi.Org,rdi):
                rdi = rdi.Rprev
            else:
                print("Finish compute the lower common tangent of L and R")
                break
        #Create a first cross edge basel from rdi.Org to ldi.Org
        basel = connect(rdi.Sym,ldi)
        if ldi.Org == ldo.Org:
            ldo = basel.Sym
        if rdi.Org == rdo.Org:
            rdo = basel
        #Merge loop
        while True:
            #Locate the first L point (lcand.Dest) to be encoutered by the rising bubble
            #and delete Ledges out of basel.Dest that fail the circle test
            lcand = basel.Sym.Onext
            if valid(lcand):
                while incircle(basel.Dest, basel.Org, lcand.Dest, lcand.Onext.Dest):
                    t = lcand.Onext
                    delete_edge(lcand)
                    lcand = t
            #End if
            #Symmetrically locate the first R point to be hit, and delete R edges
            rcand = basel.Oprev
            if valid(rcand):
                while incircle(basel.Dest, basel.Org, rcand.Dest, rcand.Oprev.Dest):
                    t = rcand.Oprev
                    delete_edge(rcand)
                    rcand = t
            #End if
            #If both lcand and rcand are invalid, then basel is the upper common tangent
            if not valid(lcand) and not valid(rcand):
                print("Both lcand and rcand are invalid")
                break
            #if both are valid, then choose the approriate one using the incircle test
            if not valid(lcand) or (valid(rcand) and incircle(lcand.Dest, lcand.Org, rcand.Org, rcand.Dest)):
                #add cross edge basel from rcand.Dest to basel.Dest
                basel = connect(rcand, basel.Sym)
            else:
                #add cross edge basel from basel.Org to lcand.Dest
                basel = connect(basel.Sym, lcand.Sym)
            #end if
        #End loop
        return (ldo, rdo)
    #End delaunay_dc 


In [36]:
# Test with 2
S = [Site(0,0),Site(1,1)]
delaunay_dc(S)

(<__main__.QuadEdge at 0x7fd110cdd0a0>, None)

In [37]:
# Test with 3
S = [Site(0,0),Site(1,1),Site(1,0)]
delaunay_dc(S)

(<__main__.QuadEdge at 0x7fd1109942e0>, None)

In [38]:
# Test with 4
S = [Site(0,0),Site(1,1),Site(1,0),Site(0,4)]
delaunay_dc(S)

AttributeError: 'NoneType' object has no attribute 'Org'

In [25]:
S[:len(S)//2]

[<__main__.Site at 0x7fd110c9ddf0>, <__main__.Site at 0x7fd110c9de50>]